In [1]:
# Clean up raw Animal Shelter intake an outcome data files downloaded from City of Austin on November 29,2017

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import datetime

In [3]:
# Set filepaths
csv_filepath1 = "raw data/Austin_Animal_Center_Intakes.csv"
csv_filepath2 = "raw data/Austin_Animal_Center_Outcomes.csv"

In [4]:
#Define reusable functions for cleanse

#Determine purebred status
def GetPurebred(x):
  if "Mix" in x["Breed"] : return 0
  elif "/" in x["Breed"] : return 0
  else: return 1


#Convert the Age to a consistent unit (days)
def GetAgeDays(x):
    if x["AgeUnits"] == "day":
        numDays = x["NumAge"]
    elif x["AgeUnits"] == "week":
        numDays = x["NumAge"] * 7
    elif x["AgeUnits"] == "month":
        numDays = x["NumAge"] * 30
    elif x["AgeUnits"] == "year":
        numDays = x["NumAge"] * 365
    else:
        numDays = 0
    return numDays


In [5]:
#  ****    Cleanse the Intake file  ****
# Read the csv files into new dataframe
intakes_df = pd.read_csv(csv_filepath1, encoding="iso-8859-1", low_memory=False)
intakes_df.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A748291,*Madison,05/01/2017 02:26:00 PM,05/01/2017 02:26:00 PM,S Pleasant Valley Rd And E Riverside Dr in Aus...,Stray,Normal,Dog,Intact Female,10 months,Pit Bull Mix,Black
1,A750529,NaN,05/28/2017 01:22:00 PM,05/28/2017 01:22:00 PM,8312 North Ih 35 in Austin (TX),Stray,Normal,Dog,Intact Female,5 months,Miniature Schnauzer Mix,White/Cream
2,A730601,NaN,07/07/2016 12:11:00 PM,07/07/2016 12:11:00 PM,1109 Shady Ln in Austin (TX),Stray,Normal,Cat,Intact Male,7 months,Domestic Shorthair Mix,Blue Tabby
3,A748238,NaN,05/01/2017 10:53:00 AM,05/01/2017 10:53:00 AM,Airport Blvd And Oak Springs Dr in Austin (TX),Stray,Normal,Dog,Intact Male,3 years,Bichon Frise Mix,White
4,A683644,*Zoey,07/13/2014 11:02:00 AM,07/13/2014 11:02:00 AM,Austin (TX),Owner Surrender,Nursing,Dog,Intact Female,4 weeks,Border Collie Mix,Brown/White


In [6]:
# Drop redundant columns
intakes_df.drop(['MonthYear'], axis = 1, inplace = True)
    #intakes_df.head()

In [7]:
# Transform date field and create new Month and Year columns
intakes_df["DateTime"] = pd.to_datetime(intakes_df["DateTime"])
intakes_df["Month"]= intakes_df["DateTime"].dt.month
intakes_df["Year"]= intakes_df["DateTime"].dt.year

In [8]:
# Identify unique values in the Sex upon Intake column
intakes_df["Sex upon Intake"].unique()

array(['Intact Female', 'Intact Male', 'Spayed Female', 'Unknown',
       'Neutered Male', nan], dtype=object)

In [9]:
# Split Sex upon Intake to an IntakeSex and IntakeSpayNeuter columns
intakes_df["Sex"] = intakes_df["Sex upon Intake"].map({"Intact Female":"Female", "Intact Male":"Male", "Spayed Female":"Female", "Neutered Male":"Male","Unknown":"Unknown Sex"})
intakes_df["Female"] = intakes_df["Sex upon Intake"].map({"Intact Female":1, "Intact Male":0, "Spayed Female":1, "Neutered Male":0,"Unknown":0})
intakes_df["Male"] = intakes_df["Sex upon Intake"].map({"Intact Female":0, "Intact Male":1, "Spayed Female":0, "Neutered Male":1,"Unknown":0})
intakes_df["SpayNeuter"] = intakes_df["Sex upon Intake"].map({"Intact Female":"No", "Intact Male":"No", "Spayed Female":"Yes", "Neutered Male":"Yes","Unknown":"No"})
    #intakes_df.head()

In [10]:
# Set Purebred Status
intakes_df['Purebred'] = intakes_df.apply(GetPurebred, axis=1)
    #intakes_df.head()

In [11]:
#Split Age Upon Intake
intakes_df['NumAge'], intakes_df['AgeUnits'] = intakes_df['Age upon Intake'].str.split(' ', 1).str
    #intakes_df.head()

In [12]:
#Make NumAge a numeric colunm for later calclations
intakes_df["NumAge"] = pd.to_numeric(intakes_df["NumAge"])
    #intakes_df.head()

In [13]:
# Identify unique values in the intake age units
intakes_df["AgeUnits"].unique()

array(['months', 'years', 'weeks', 'month', 'year', 'week', 'days', 'day',
       nan], dtype=object)

In [14]:
#Make the intake age unit values consistent
intakes_df["AgeUnits"].replace(["days","weeks","months","years"],["day","week","month","year"], inplace=True)
    #intakes_df["IntakeAgeUnits"].unique()

In [15]:
#Set the Age Days column
intakes_df['AgeDays'] = intakes_df.apply(GetAgeDays, axis=1)
    #intakes_df.head()

In [16]:
# Get metrics on the raw intake file
intakes_df.describe()

,Month,Year,Female,Male,Purebred,NumAge,AgeDays
count,75577.000000,75577.000000,75576.000000,75576.000000,75577.000000,75576.000000,75577.000000
mean,6.689601,2015.309009,0.437388,0.476540,0.067322,3.445300,761.731559
std,3.257848,1.213225,0.496067,0.499453,0.250581,2.908907,1052.134727
min,1.000000,2013.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,2014.000000,0.000000,0.000000,0.000000,1.000000,60.000000
50%,7.000000,2015.000000,0.000000,0.000000,0.000000,2.000000,365.000000
75%,10.000000,2016.000000,1.000000,1.000000,0.000000,5.000000,1095.000000
max,12.000000,2017.000000,1.000000,1.000000,1.000000,25.000000,9125.000000


In [17]:
#Metrics cont'd
intake_recs = len(intakes_df)
intake_dups = intakes_df.duplicated("Animal ID").sum()
intake_types = intakes_df.groupby(["Animal Type"]).size()

print(f"Number of records in Intakes file: {intake_recs}")
print(f"Duplicate Intake Animal IDs: {intake_dups}")
print(f"Intakes by Animal Type: {intake_types}")

Number of records in Intakes file: 75577
Duplicate Intake Animal IDs: 7031
Intakes by Animal Type: Animal Type
Bird           328
Cat          28489
Dog          42590
Livestock        8
Other         4162
dtype: int64


In [18]:
#  *** Cleanse the Outcomes file  ***
# Read the csv files into new dataframes
outcomes_df = pd.read_csv(csv_filepath2, encoding="iso-8859-1", low_memory=False)
outcomes_df.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A741715,*Pebbles,01/11/2017 06:17:00 PM,01/11/2017 06:17:00 PM,03/07/2016,Adoption,NaN,Cat,Spayed Female,10 months,Domestic Shorthair Mix,Calico
1,A658751,Benji,11/13/2016 01:38:00 PM,11/13/2016 01:38:00 PM,07/14/2011,Return to Owner,NaN,Dog,Neutered Male,5 years,Border Terrier Mix,Tan
2,A721285,NaN,02/24/2016 02:42:00 PM,02/24/2016 02:42:00 PM,02/24/2014,Euthanasia,Suffering,Other,Unknown,2 years,Raccoon Mix,Black/Gray
3,A746650,Rose,04/07/2017 11:58:00 AM,04/07/2017 11:58:00 AM,04/06/2016,Return to Owner,NaN,Dog,Intact Female,1 year,Labrador Retriever/Jack Russell Terrier,Yellow
4,A750122,Happy Camper,05/24/2017 06:36:00 PM,05/24/2017 06:36:00 PM,04/08/2017,Transfer,Partner,Dog,Intact Male,1 month,Labrador Retriever Mix,Black


In [19]:
# Drop redundant columns
outcomes_df.drop(['MonthYear'], axis = 1, inplace = True)
    #outcomes_df.head()

In [20]:
# Transform date field and create new Month and Year columns
outcomes_df["DateTime"] = pd.to_datetime(outcomes_df["DateTime"])
outcomes_df["Month"]= outcomes_df["DateTime"].dt.month
outcomes_df["Year"]= outcomes_df["DateTime"].dt.year

In [21]:
# Identify unique values in the Sex upon Outcome column
outcomes_df["Sex upon Outcome"].unique()

array(['Spayed Female', 'Neutered Male', 'Unknown', 'Intact Female',
       'Intact Male', nan], dtype=object)

In [22]:
# Split Sex upon Outcome to an OutcomeSex and OutcomeSpayNeuter columns
outcomes_df["Sex"] = outcomes_df["Sex upon Outcome"].map({"Intact Female":"Female", "Intact Male":"Male", "Spayed Female":"Female", "Neutered Male":"Male","Unknown":"Unknown Sex"})
outcomes_df["Female"] = outcomes_df["Sex upon Outcome"].map({"Intact Female":1, "Intact Male":0, "Spayed Female":1, "Neutered Male":0,"Unknown":0})
outcomes_df["Male"] = outcomes_df["Sex upon Outcome"].map({"Intact Female":0, "Intact Male":1, "Spayed Female":0, "Neutered Male":1,"Unknown":0})
outcomes_df["SpayNeuter"] = outcomes_df["Sex upon Outcome"].map({"Intact Female":"No", "Intact Male":"No", "Spayed Female":"Yes", "Neutered Male":"Yes","Unknown":"No"})
    #outcomes_df.head()

In [23]:
# Get purebred 
outcomes_df['Purebred'] = outcomes_df.apply(GetPurebred, axis=1)

In [24]:
# Split out age
outcomes_df['NumAge'], outcomes_df['AgeUnits'] = outcomes_df['Age upon Outcome'].str.split(' ', 1).str

#Make NumAge a numeric colunm for later calclations
outcomes_df["NumAge"] = pd.to_numeric(outcomes_df["NumAge"])

#Make the intake age unit values consistent
outcomes_df["AgeUnits"].replace(["days","weeks","months","years"],["day","week","month","year"], inplace=True)

#Set the Age Days column
outcomes_df['AgeDays'] = outcomes_df.apply(GetAgeDays, axis=1)

In [25]:
# Get metrics on the raw outcomes file
outcomes_df.describe()

,Month,Year,Female,Male,Purebred,NumAge,AgeDays
count,75508.000000,75508.000000,75506.000000,75506.000000,75508.000000,75502.000000,75508.000000
mean,6.799902,2015.293770,0.437767,0.476492,0.067198,3.528211,772.159175
std,3.301991,1.219788,0.496115,0.499450,0.250367,2.901128,1052.357298
min,1.000000,2013.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,2014.000000,0.000000,0.000000,0.000000,2.000000,90.000000
50%,7.000000,2015.000000,0.000000,0.000000,0.000000,2.000000,365.000000
75%,10.000000,2016.000000,1.000000,1.000000,0.000000,5.000000,1095.000000
max,12.000000,2017.000000,1.000000,1.000000,1.000000,25.000000,9125.000000


In [26]:
#Metrics cont'd
outcome_recs = len(outcomes_df)
outcome_dups = outcomes_df.duplicated("Animal ID").sum()
outcome_types = outcomes_df.groupby(["Animal Type"]).size()

print(f"Number of records in Outcomes file: {outcome_recs}")
print(f"Duplicate Outcome Animal IDs: {outcome_dups}")
print(f"Outcomes by Animal Type: {outcome_types}")

Number of records in Outcomes file: 75508
Duplicate Outcome Animal IDs: 6975
Outcomes by Animal Type: Animal Type
Bird           327
Cat          28519
Dog          42498
Livestock        9
Other         4155
dtype: int64


In [27]:
# **** Merge intakes and outcomes files
animalData_df = pd.merge(intakes_df, outcomes_df, on="Animal ID", how="inner", suffixes=('_intake', '_outcome'))
animalData_df.head()

,Animal ID,Name_intake,DateTime_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,...,Month_outcome,Year_outcome,Sex_outcome,Female_outcome,Male_outcome,SpayNeuter_outcome,Purebred_outcome,NumAge_outcome,AgeUnits_outcome,AgeDays_outcome
0,A748291,*Madison,2017-05-01 14:26:00,S Pleasant Valley Rd And E Riverside Dr in Aus...,Stray,Normal,Dog,Intact Female,10 months,Pit Bull Mix,...,9,2017,Female,1.0,0.0,Yes,0,1.0,year,365.0
1,A750529,NaN,2017-05-28 13:22:00,8312 North Ih 35 in Austin (TX),Stray,Normal,Dog,Intact Female,5 months,Miniature Schnauzer Mix,...,6,2017,Female,1.0,0.0,Yes,0,5.0,month,150.0
2,A730601,NaN,2016-07-07 12:11:00,1109 Shady Ln in Austin (TX),Stray,Normal,Cat,Intact Male,7 months,Domestic Shorthair Mix,...,7,2016,Male,0.0,1.0,Yes,0,7.0,month,210.0
3,A748238,NaN,2017-05-01 10:53:00,Airport Blvd And Oak Springs Dr in Austin (TX),Stray,Normal,Dog,Intact Male,3 years,Bichon Frise Mix,...,5,2017,Male,0.0,1.0,Yes,0,3.0,year,1095.0
4,A683644,*Zoey,2014-07-13 11:02:00,Austin (TX),Owner Surrender,Nursing,Dog,Intact Female,4 weeks,Border Collie Mix,...,11,2014,Female,1.0,0.0,Yes,0,4.0,month,120.0


In [28]:
print(animalData_df.columns)

Index(['Animal ID', 'Name_intake', 'DateTime_intake', 'Found Location',
       'Intake Type', 'Intake Condition', 'Animal Type_intake',
       'Sex upon Intake', 'Age upon Intake', 'Breed_intake', 'Color_intake',
       'Month_intake', 'Year_intake', 'Sex_intake', 'Female_intake',
       'Male_intake', 'SpayNeuter_intake', 'Purebred_intake', 'NumAge_intake',
       'AgeUnits_intake', 'AgeDays_intake', 'Name_outcome', 'DateTime_outcome',
       'Date of Birth', 'Outcome Type', 'Outcome Subtype',
       'Animal Type_outcome', 'Sex upon Outcome', 'Age upon Outcome',
       'Breed_outcome', 'Color_outcome', 'Month_outcome', 'Year_outcome',
       'Sex_outcome', 'Female_outcome', 'Male_outcome', 'SpayNeuter_outcome',
       'Purebred_outcome', 'NumAge_outcome', 'AgeUnits_outcome',
       'AgeDays_outcome'],
      dtype='object')


In [29]:
#Remove rows with duplicate animal IDs
animalData_df=animalData_df.drop_duplicates("Animal ID",keep=False)
len(animalData_df)

62049

In [30]:
#Test to see if any other columns are redundant
animalData_df["Color_intake"].equals(animalData_df["Color_outcome"])

False

In [31]:
#Filter for only Dogs and Cats
animalData_df = animalData_df.loc[animalData_df["Animal Type_intake"].isin(["Dog","Cat"])]
len(animalData_df)

57593

In [32]:
# Calc date diff between intakes and outcomes
animalData_df["Intake Outcome Days"] = animalData_df["DateTime_outcome"]-animalData_df["DateTime_intake"]
animalData_df.head()

,Animal ID,Name_intake,DateTime_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,...,Year_outcome,Sex_outcome,Female_outcome,Male_outcome,SpayNeuter_outcome,Purebred_outcome,NumAge_outcome,AgeUnits_outcome,AgeDays_outcome,Intake Outcome Days
0,A748291,*Madison,2017-05-01 14:26:00,S Pleasant Valley Rd And E Riverside Dr in Aus...,Stray,Normal,Dog,Intact Female,10 months,Pit Bull Mix,...,2017,Female,1.0,0.0,Yes,0,1.0,year,365.0,125 days 09:34:00
1,A750529,NaN,2017-05-28 13:22:00,8312 North Ih 35 in Austin (TX),Stray,Normal,Dog,Intact Female,5 months,Miniature Schnauzer Mix,...,2017,Female,1.0,0.0,Yes,0,5.0,month,150.0,4 days 03:20:00
2,A730601,NaN,2016-07-07 12:11:00,1109 Shady Ln in Austin (TX),Stray,Normal,Cat,Intact Male,7 months,Domestic Shorthair Mix,...,2016,Male,0.0,1.0,Yes,0,7.0,month,210.0,0 days 20:49:00
3,A748238,NaN,2017-05-01 10:53:00,Airport Blvd And Oak Springs Dr in Austin (TX),Stray,Normal,Dog,Intact Male,3 years,Bichon Frise Mix,...,2017,Male,0.0,1.0,Yes,0,3.0,year,1095.0,5 days 05:04:00
4,A683644,*Zoey,2014-07-13 11:02:00,Austin (TX),Owner Surrender,Nursing,Dog,Intact Female,4 weeks,Border Collie Mix,...,2014,Female,1.0,0.0,Yes,0,4.0,month,120.0,115 days 23:04:00


In [33]:
#Export to CSV
animalData_df.to_csv("raw data/MergedData.csv", encoding="utf-8", index=False)

In [34]:
# Get metrics on the raw outcomes file
animalData_df.describe()

,Month_intake,Year_intake,Female_intake,Male_intake,Purebred_intake,NumAge_intake,AgeDays_intake,Month_outcome,Year_outcome,Female_outcome,Male_outcome,Purebred_outcome,NumAge_outcome,AgeDays_outcome,Intake Outcome Days
count,57593.000000,57593.000000,57592.000000,57592.000000,57593.000000,57593.000000,57593.000000,57593.000000,57593.000000,57591.000000,57591.000000,57593.000000,57588.000000,57593.000000,57593
mean,6.708697,2015.260709,0.462425,0.490294,0.048947,3.457382,742.241818,6.850589,2015.294480,0.462416,0.490302,0.048947,3.557599,756.069627,16 days 21:59:22.331880
std,3.210898,1.210101,0.498590,0.499910,0.215759,2.936187,1087.364762,3.262525,1.207164,0.498590,0.499910,0.215759,2.929233,1089.431333,37 days 09:35:10.776463
min,1.000000,2013.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2013.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1 days +03:18:00
25%,4.000000,2014.000000,0.000000,0.000000,0.000000,1.000000,30.000000,4.000000,2014.000000,0.000000,0.000000,0.000000,2.000000,60.000000,1 days 08:43:00
50%,7.000000,2015.000000,0.000000,0.000000,0.000000,2.000000,365.000000,7.000000,2015.000000,0.000000,0.000000,0.000000,2.000000,365.000000,5 days 02:43:00
75%,9.000000,2016.000000,1.000000,1.000000,0.000000,4.000000,730.000000,10.000000,2016.000000,1.000000,1.000000,0.000000,5.000000,730.000000,14 days 06:39:00
max,12.000000,2017.000000,1.000000,1.000000,1.000000,22.000000,8030.000000,12.000000,2017.000000,1.000000,1.000000,1.000000,22.000000,8030.000000,1101 days 18:49:00


In [35]:
#Metrics cont'd
data_recs = len(animalData_df)
data_AnimalTypes = animalData_df.groupby(["Animal Type_intake"]).size()
data_AnimalSex = animalData_df.groupby(["Animal Type_intake", "Sex_intake"]).size()
data_AnimalIntake = animalData_df.groupby(["Animal Type_intake", "Intake Type"]).size()
data_AnimalOutcome = animalData_df.groupby(["Animal Type_intake", "Outcome Type"]).size()

print(f"Number of records in Outcomes file: {data_recs}")
print(f"Number of Animal Type: {data_AnimalTypes}")
print(f"Number of Animals by Type and Sex: {data_AnimalSex}")
print(f"Number of Animals by Type and Intake Reason: {data_AnimalIntake}")
print(f"Number of Animals by Type and Outcome: {data_AnimalOutcome}")

Number of records in Outcomes file: 57593
Number of Animal Type: Animal Type_intake
Cat    25896
Dog    31697
dtype: int64
Number of Animals by Type and Sex: Animal Type_intake  Sex_intake 
Cat                 Female         11909
                    Male           11474
                    Unknown Sex     2513
Dog                 Female         14723
                    Male           16763
                    Unknown Sex      210
dtype: int64
Number of Animals by Type and Intake Reason: Animal Type_intake  Intake Type       
Cat                 Euthanasia Request       50
                    Owner Surrender        4313
                    Public Assist           558
                    Stray                 20975
Dog                 Euthanasia Request      169
                    Owner Surrender        4913
                    Public Assist          2736
                    Stray                 23879
dtype: int64
Number of Animals by Type and Outcome: Animal Type_intake  Outcome Typ